# Random Forest Classifier

In [1]:
import pandas as pd
import numpy as np
import os
import json

from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder, PolynomialFeatures
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import cross_val_score, cross_val_predict, cross_validate

from sklearn.dummy import DummyClassifier, DummyRegressor
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import make_scorer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix  # Recommended method in sklearn 1.0
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

In [2]:
data_path = "../data/processed"

#Load data mappings
mappings = json.load(open("data_mapping.json", "r"))

train_df = pd.read_csv(os.path.join(data_path, "train.csv"))
test_df = pd.read_csv(os.path.join(data_path, "test.csv"))

X_train = train_df.drop(columns=mappings['drugs'])
y_train = train_df[mappings['drugs']]
X_test = test_df.drop(columns=mappings['drugs'])
y_test = test_df[mappings['drugs']]


In [3]:
#Scoring functions to use
scorers = {
            'f1_score': make_scorer(f1_score, average='weighted', zero_division=True),
            # 'precision_score': make_scorer(precision_score, average='weighted',zero_division=True),
            # 'recall_score': make_scorer(recall_score, average='weighted', zero_division=True),
            # 'accuracy_score': make_scorer(accuracy_score)
          }

In [4]:
def mean_std_cross_val_scores(model, X_train, y_train, **kwargs):
    """
    Returns mean and std of cross validation

    Parameters
    ----------
    model :
        scikit-learn model
    X_train : numpy array or pandas DataFrame
        X in the training data
    y_train :
        y in the training data

    Returns
    ----------
        pandas Series with mean scores from cross_validation
    """

    scores = cross_validate(model, X_train, y_train, **kwargs)
    mean_scores = pd.DataFrame(scores).round(4).mean()
    std_scores = pd.DataFrame(scores).round(4).std()
    out_col = []

    for i in range(len(mean_scores)):
        out_col.append((f"%0.3f (+/- %0.3f)" % (mean_scores[i], std_scores[i])))

    return pd.Series(data=out_col, index=mean_scores.index)

In [5]:
results = {}
dummy_cv_results = {}
# Get the mean accuracy for each drug
for drug in mappings['drugs']: 
    results[drug] = {}
    dc = DummyClassifier(strategy="most_frequent")
    # dc.fit(X_train, y_train[drug])
    dummy_cv_results[drug] = mean_std_cross_val_scores( dc, 
                                                        X_train,
                                                        y_train[drug],
                                                        return_train_score = True,
                                                        n_jobs=-1,
                                                        scoring=scorers) 
results = pd.DataFrame(dummy_cv_results)
results = results.drop(index = ["fit_time", "score_time"]).T
results = results.reset_index()
results = results.rename(columns = {"index": "target_drug"})

results = results.set_index("target_drug").T

In [6]:
results

target_drug,Alcohol,Cannabis,Chocolate,Caffeine,Cocaine,Mushrooms,Nicotine
test_f1_score,0.542 (+/- 0.003),0.305 (+/- 0.002),0.701 (+/- 0.002),0.835 (+/- 0.003),0.681 (+/- 0.002),0.675 (+/- 0.003),0.263 (+/- 0.002)
train_f1_score,0.542 (+/- 0.001),0.305 (+/- 0.001),0.701 (+/- 0.000),0.835 (+/- 0.001),0.681 (+/- 0.001),0.675 (+/- 0.001),0.263 (+/- 0.001)


In [7]:
rf_pipe =  make_pipeline(
    # preprocessor,
    RandomForestClassifier(min_samples_leaf=2,
                           n_jobs=-1,
                           random_state=522)
)

param_dist = {
    "randomforestclassifier__max_depth": [i for i in range(3, 35, 2)],
    "randomforestclassifier__max_features": [i for i in range(6, 15, 2)],
    "randomforestclassifier__class_weight": ["balanced", "balanced_subsample", None],
    "randomforestclassifier__n_estimators": [i for i in range(50, 250, 10)]
}

# Save the best model and score for each drug
rf_best_estimator = {}
rf_best_score_by_drug = {}

for drug in mappings['drugs']: 
    random_search = RandomizedSearchCV(rf_pipe, 
                                param_distributions=param_dist,
                                n_jobs = -1,
                                n_iter=3,
                                cv = 3, 
                                return_train_score = True,
                                scoring=scorers,
                                refit='f1_score')
    
    random_search.fit(X_train, y_train[drug])
    rf_best_estimator[drug] = random_search.best_estimator_
    
    rf_best_score_by_drug[drug] = mean_std_cross_val_scores(random_search,
                              X_train,
                              y_train[drug],
                              cv=3,
                              n_jobs=-1,
                              return_train_score=True)
    
    
rf_score_by_drug = pd.DataFrame(rf_best_score_by_drug)
rf_score_by_drug = rf_score_by_drug.drop(index = ["fit_time", "score_time"]).T
rf_score_by_drug = rf_score_by_drug.reset_index()
rf_score_by_drug = rf_score_by_drug.rename(columns = {"index": "target_drug"})
rf_score_by_drug = rf_score_by_drug.set_index("target_drug").T
rf_score_by_drug.index = ['rf_test_f1_score', 'rf_train_f1_score']

    
# results = pd.concat([results, rf_score_by_drug])

In [8]:
rf_score_by_drug

target_drug,Alcohol,Cannabis,Chocolate,Caffeine,Cocaine,Mushrooms,Nicotine
rf_test_f1_score,0.575 (+/- 0.026),0.615 (+/- 0.011),0.706 (+/- 0.003),0.835 (+/- 0.000),0.711 (+/- 0.017),0.766 (+/- 0.015),0.543 (+/- 0.001)
rf_train_f1_score,1.000 (+/- 0.000),0.992 (+/- 0.007),1.000 (+/- 0.001),0.932 (+/- 0.018),0.967 (+/- 0.031),0.976 (+/- 0.038),0.905 (+/- 0.111)
